In [1]:
## define all binaries
mk_prepapre_ligand = '/Users/vbf/Library/Python/3.9/bin/mk_prepare_ligand.py'
vina = '/Users/vbf/Work/bin/vina_1.2.7_mac_aarch64'
obabel = 'obabel'

In [2]:
ligand_file = '0G6_H'
!obabel /Users/vbf/Work/projects/terraq/1ppb/vina/2_redock/1_ligand/1ppb_C_0G6.sdf -O{ligand_file}.sdf -ph 7.4

1 molecule converted


In [3]:
!{mk_prepapre_ligand} -i {ligand_file}.sdf -o {ligand_file}.pdbqt

Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0


In [ ]:
exh_vals = [8*2**i for i in range(5)]

receptor_file = '1ppb_receptor'

for i in exh_vals:
    cmd = (f"{vina} --ligand {ligand_file}.pdbqt  --receptor {receptor_file}.pdbqt "
       f"--config {receptor_file}.box.txt --exhaustiveness={i} --out 2_redock/2_docking/docked{i}.pdb "
        f"--num_modes 9 | tee 2_redock/2_docking/docking{i}.log")
    print(f"Running with exhaustiveness {i}")
    print(cmd)
    !{cmd}

Running with exhaustiveness 8
/Users/vbf/Work/bin/vina_1.2.7_mac_aarch64 --ligand 0G6_H.pdbqt  --receptor 1ppb_receptor.pdbqt --config 1ppb_receptor.box.txt --exhaustiveness=8 --out 2_redock/2_docking/docked8.pdb --num_modes 9 | tee 2_redock/2_docking/docking8.log
tee: 2_redock/2_docking/docking8.log: No such file or directory
AutoDock Vina v1.2.7
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of doc

In [ ]:
import re
from rdkit import Chem
from rdkit.Chem import AllChem

def get_first_mol_from_pdbqt(file):
    mol_block = ""
    score = None
    with open(file, 'r') as f:
        for i, line in enumerate(f):
            if i == 1:
                finds = re.findall(r"REMARK VINA RESULT:\s+([0-9\-.]+)", line)
                if finds != []:
                    score = float(finds[0])

            if line.startswith("ATOM"):
                line_ = line.strip()
                mol_block += line_[:len(line_)-2]+'\n'

            if line.strip() == 'ENDMDL':
                break

    return Chem.MolFromPDBBlock(mol_block, removeHs=True), score

mol_original, _ = get_first_mol_from_pdbqt("2_redock/1_ligand/0G6_H.pdbqt")

def get_scores(file):
    mol,score = get_first_mol_from_pdbqt(file)
    return AllChem.GetBestRMS(mol_original, mol), score

In [ ]:
file_list = [f"2_redock/2_docking/docked{i}.pdb" for i in exh_vals]
dock_scores = []
rmsd_scores = []

for file in file_list:
    i,j = get_scores(file)
    rmsd_scores.append(i)
    dock_scores.append(j)